In [1]:
import pandas as pd
from pymongo import MongoClient
import config as cfg

In [2]:
# 몽고DB 접속 정보로 크롤링된 데이터 불러오기
mongo = MongoClient(cfg.mongo_DB_HOST, cfg.mongo_DB_PORT)
news = mongo.myarticles.articles

result = []
for n in news.find():
    result.append({ 'id' : n['_id'],
                    'title': n['title'],
                    'content' : n['content']})

In [3]:
data = pd.DataFrame(result)

In [4]:
data.shape

(369, 3)

In [5]:
data.tail()

,content,id,title
364,내달 12일 코엑스서 기업 초청 사업화 설명회(대전=연합뉴스) 박주영 기자 = 음주...,http://news.naver.com/main/read.nhn?mode=LSD&m...,음주측정기처럼 '훅' 불어 질병 진단…카이스트 10대 기술 선정
365,[CBS노컷뉴스 이재웅 기자](사진=자료사진)최근 엔씨소프트의 모바일 게임 리니지M...,http://news.naver.com/main/read.nhn?mode=LSD&m...,'리니지M' 환불거부 배짱영업 주의보
366,지난주 2천여명이 참석했던 파이썬 컨퍼런스 ‘파이콘’이 성황리에 끝났다. 60여개의...,http://news.naver.com/main/read.nhn?mode=LSD&m...,'얼리또라이’의 데이터 공부 도전기
367,도메인등록 전문업체 고대디(GoDaddy) CEO 블레이크 어빙이 은퇴를 선언했다....,http://news.naver.com/main/read.nhn?mode=LSD&m...,"블레이크 어빙 고대디 CEO 은퇴, 후임자는 스캇 와그너"
368,마이크로스파인의 그립식 발 탑재(지디넷코리아=이정현 기자)넓고 평평한 지표면이 아닌...,http://news.naver.com/main/read.nhn?mode=LSD&m...,곤충처럼 벽에 달라붙어 착륙하는 드론


In [6]:
%%time
# 불러온 데이터 중에서 기사 본문을 읽어서 텍스트 문서로 저장하기
USE_PREMADE_CONTENT = False

from os import path
content_filepath = 'task/content.txt'

if not USE_PREMADE_CONTENT:
    
    with open(content_filepath, 'w', encoding='utf-8') as f:
        for article in data.content.values:
            # 기사가 없으면 스킵
            if pd.isnull(article):
                continue
            f.write(article + '\n')
else:
    assert path.exists(content_filepath)

Wall time: 8.01 ms


In [7]:
# 저장된 텍스트 문서에서 기사 읽어오는 함수
def read_article(filepath):
    with open(filepath, encoding='utf-8') as f:
        for article in f:
            yield article

In [8]:
# 읽어온 모든 기사를 기사별로 나눠주는 함수. 숫자로 기사 불러오기.
from itertools import islice

def retrieve_article(sample_num):
    return next(islice(read_article(content_filepath), sample_num, sample_num+1))

In [9]:
# 한글 형태소 분석기. konlpy Twitter 사용
from konlpy.tag import Twitter
tw = Twitter()

In [10]:
# stop_words : 걸러내야 하는 단어 모음.
# stop_words = open('intermediate/stop_words.txt').read().split('\n')
stop_words = []

# 텍스트 정규화 : 여기서는 명사만 추출하고 추가적으로 정규화할 때 stop_words를 사용
def normalize(text):
    nouns = tw.nouns(text)
    filtered_nouns = [ noun for noun in nouns if len(noun) > 1 and noun not in stop_words]
    return filtered_nouns

In [11]:
%%time
# 저장된 기사의 텍스트를 정규화하여 저장하기
USE_PREMADE_SENTENCES_NORMALIZED = False

sentences_normalized_filepath = 'task/contents_normalized.txt'

if not USE_PREMADE_SENTENCES_NORMALIZED:
    
    with open(sentences_normalized_filepath, 'w',  encoding='utf-8') as f:
        for article_parsed in read_article(content_filepath):
            sentence_parsed = normalize(article_parsed)
            f.write(' '.join(sentence_parsed) + '\n')
            
else:
    assert path.exists(sentences_normalized_filepath)

Wall time: 11.7 s


In [12]:
# gensim 라이브러리에서 Phrases 클래스 로딩
from gensim.models import Phrases

# 텍스트파일에서 line 단위로 iterate 하는 LineSentence 클래스 제공
# 한번에 하나의 line을 출력(generator)
from gensim.models.word2vec import LineSentence

c:\Python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [13]:
%%time
USE_PREMADE_BIGRAM_MODEL = False

bigram_model_filepath = 'task/bigram_model'

# 정규화된 텍스트를 input으로 받아서 해당 객체 형태로 출력한다.
unigram_sentences = LineSentence(sentences_normalized_filepath)

if not USE_PREMADE_BIGRAM_MODEL:
    
    # 출력된 객체 Phrases 클래스에 통과시켜서 bigram model 생성.
    # bigram은 같이 등장하는 두개의 단어를 하나의 어구로 보게 하기 위함
    bigram_model = Phrases(unigram_sentences)
    bigram_model.save(bigram_model_filepath)
    
else:
    bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 279 ms


In [14]:
%%time
# 생성된 bigram model에 Linesentence을 통과 시킨 객채를 넣어 bigram 텍스트 생성
USE_PREMADE_BIGRAM_SENTENCES = False

bigram_sentences_filepath = 'task/bigram_sentences.txt'

if not USE_PREMADE_BIGRAM_SENTENCES:
    
    with open(bigram_sentences_filepath, 'w', encoding='utf-8') as f:
        for unigram_sentence in unigram_sentences:
            bigram_sentence = bigram_model[unigram_sentence]
            f.write(' '.join(bigram_sentence) + '\n')
else:
    assert path.exists(bigram_sentences_filepath)

c:\Python36\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 351 ms


In [15]:
%%time
# n-gram을 한번 더 통과시켜서 3개 어구짜리 모델 생성
USE_PREMADE_TRIGRAM_MODEL = False

trigram_model_filepath = 'task/trigram_model'

from gensim.models.word2vec import LineSentence
from gensim.models import Phrases

if not USE_PREMADE_TRIGRAM_MODEL:
    
    bigram_sentences = LineSentence(bigram_sentences_filepath)
    trigram_model = Phrases(bigram_sentences)
    trigram_model.save(trigram_model_filepath)

else:
    trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 272 ms


In [16]:
%%time
# 3개 어구까지 만들고 텍스트 파일로 저장. lda 모델에 사용하기 위한 마지막 단계
USE_PREMADE_CONTENTS_FOR_LDA = False

articles_for_lda_filepath = 'task/contents_for_lda.txt'

if not USE_PREMADE_CONTENTS_FOR_LDA:
    
    with open(articles_for_lda_filepath, 'w', encoding='utf-8') as f:
        
        for article_parsed in read_article(content_filepath):
            
            unigram_article = normalize(article_parsed)
            bigram_article = bigram_model[unigram_article]
            trigram_article = trigram_model[bigram_article]
            f.write(' '.join(trigram_article) + '\n')
else:
    assert path.exists(contents_for_lda_filepath)

c:\Python36\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 10.7 s


In [17]:
# Dictionary와 Corpus 클래스 로딩
# Dictionary는 특정 단어에 id를 부여하기 위해 사용
# Corpus는 말뭉치로 어느 단어가 몇회 등장하나
from gensim.corpora import Dictionary, MmCorpus

In [18]:
%%time
# 가지고 있는 텍스트로 dictionary 생성
USE_PREMADE_DICTIONARY = False

dictionary_filepath = 'task/dictionary.dict'

if not USE_PREMADE_DICTIONARY:
    
    articles_for_lda = LineSentence(articles_for_lda_filepath)
    dictionary = Dictionary(articles_for_lda)
    #dictionary.filter_extremes(no_below=3, no_above=0.3)
    dictionary.compactify()
    
    dictionary.save(dictionary_filepath)
else:
    dictionary = Dictionary.load(dictionary_filepath)

Wall time: 130 ms


In [19]:
%%time
# Corpus 생성
USE_PREMADE_CORPUS = False

corpus_filepath = 'task/corpus.mm'

if not USE_PREMADE_CORPUS:
    
    def make_bow_corpus(filepath):
        # 정규화된 기사를 읽어와서 bag of words로 만드는 함수.
        # bag of words : 순서나 문맥과 관련 없는 단어의 집합
        for article in LineSentence(filepath):
            yield dictionary.doc2bow(article)
            
    MmCorpus.serialize(corpus_filepath, make_bow_corpus(articles_for_lda_filepath))
    
article_corpus = MmCorpus(corpus_filepath)

Wall time: 172 ms


In [20]:
# LDA 모델 로딩
from gensim.models import LdaMulticore

In [21]:
%%time
# corpus와 dictionary를 넣고, cpu 개수에 따라 worker 개수 설정하여 병렬처리,
# num_topics 는 주제의 개수 설정, passes 는 전체 단어 학습 횟수
# 위 인자 등을 가지고 lda 모델 생성
USE_PREMADE_LDA = False

lda_filepath = 'task/lda'

if not USE_PREMADE_LDA:
    
    lda = LdaMulticore(article_corpus,
                           num_topics=110,
                           id2word=dictionary,
                           workers=3,
                           passes=200)
    lda.save(lda_filepath)
else:
    lda = LdaMulticore.load(lda_filepath)

Wall time: 13min 14s


In [ ]:
# import pyLDAvis
# import pyLDAvis.gensim
# import pickle

In [ ]:
# %%time
# USE_PREMADE_LDAVIS = False

# ldavis_filepath = 'medium/ldavis'

# if not USE_PREMADE_LDAVIS:
#     ldavis = pyLDAvis.gensim.prepare(topic_model=lda, 
#                                      corpus=review_corpus, 
#                                      dictionary=dictionary)
    
#     with open(ldavis_filepath, 'wb') as f:
#         pickle.dump(ldavis, f)

# else:
#     with open(ldavis_filepath, 'rb') as f:
#         ldavis = pickle.load(f)

In [ ]:
# pyLDAvis.display(ldavis)

In [22]:
# 생성된 lda를 가지고 각 기사에 대한 topic을 추출하는 함수. 선택한 topic 개수에 따라 번호로 주제명이 부여됨.
def get_article_lda(article):
    article_lemmatized = normalize(article)
    article_bigram = bigram_model[article_lemmatized]
    article_trigram = trigram_model[article_bigram]
    article_bow = dictionary.doc2bow(article_trigram)
    article_topic = lda[article_bow]
    
    return article_topic

In [23]:
# get_article_lda 로 뽑힌 주제중 각 주제에서 자주 등장하는 단어중 빈도가 제일 높은 단어 선택 -> 해당 주제의 주제로 선정하기 위해 사용
def inner_topic(topic_num):
    return sorted(lda.show_topic(topic_num), key=lambda x:x[1], reverse=True)[0][0]

In [24]:
# 주제 번호로 해당 주제를 나타내는 단어 찾기
def get_topic_name(topic):
    result = []
    for x in topic:
        result.append((x[0], inner_topic(x[0])))
    return result

In [25]:
# 각 기사의 번호를 가지고 해당 기사의 url과 제목, 주제 가지고 오는 함수
# '' 값으로 처리된 rel1~7은 차후에 보다 검색 결과의 정확도를 높이기 위해서 미리 만들어 놓은 변수.
def assign_topic2data(article_num):
    single_article = retrieve_article(article_num)
    rel1 = ''
    rel2 = ''
    rel3 = ''
    rel4 = ''
    rel5 = ''
    rel6 = ''
    rel7 = ''
    topic = get_article_lda(single_article)
    return (data.iloc[article_num].id, data.iloc[article_num].title, get_topic_name(topic)[0][1], rel1, rel2, rel3, rel4, rel5, rel6, rel7)

In [26]:
# Mssql db 저장 모듈 로딩
from newsdao import NewsDAO
newsdao = NewsDAO()

In [27]:
%%time
# assign_topic2data 함수 호출하여 결과 값을 받아서 db에 저장
for i in range(len(data)):
    url, title, topic, rel1, rel2, rel3, rel4, rel5, rel6, rel7 = assign_topic2data(i)
    newsdao.save_news(url, title, topic, rel1, rel2, rel3, rel4, rel5, rel6, rel7)

c:\Python36\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
